### 必要なパッケージをインポート

#### .envファイル
```yaml
OPENAI_API_KEY=sk-************************
```

In [1]:
import openai,os,requests,json,re
from tqdm import tqdm
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import UnstructuredHTMLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI

### Prompts

LangChainのRefineに倣い、InitialプロンプトとRefineプロンプトを作成した。\
不要な「,」を含んで、json.loads()でパースできない場合があったため、以下を追加した。\
（これでもたまにパースできないJsonを返してくる。。\
LangChainのOutput Parserを使いたいが、リストや入れ子構造があるJsonに対する方法がわからない。\
Pydantic Parserだと、英語で書かないとプロンプトが文字化けする。）

```
json.loads()関数でパースできるよう、listやdictの最後の要素には「,」を含めないでください。
```

また、Refineプロンプトでは、必要な場合のみ修正するようにした。
ドキュメントの最初に重要な情報が多く、最後に近づくにつれ関連コンテンツなど、メインコンテンツと関係ないものが多くなるという仮定を入れている。

```
[Text]をもとに、[Data]について新しい情報がある場合のみ[Data]を修正してください。\
[Text]が[Data]に関連した情報を提供しない場合、[Data]をそのまま出力してください。
```

In [2]:
initial_str = """\
Output Format:
```json
{output_format}
```

Input:
{text}

Instruction:
[Input]はあるWebページのテキストの一部です。\
[Output Format]に従って出力してください。\
json.loads()関数でパースできるよう、listやdictの最後の要素には「,」を含めないでください。\
[Input]から読み取れないデータは出力に含めないでください。
"""

refine_str = """\
Output Format:
```json
{output_format}
```

Data:
{data}

Text:
{text}

## Instruction
あなたのタスクは、最終的なデータを生成することです。\
現時点でのデータは[Data]です。
[Text]をもとに、[Data]について新しい情報がある場合のみ[Data]を修正してください。\
[Text]が[Data]に関連した情報を提供しない場合、[Data]をそのまま出力してください。
[Output Format]に従って出力してください。\
json.loads()関数でパースできるよう、listやdictの最後の要素には「,」を含めないでください。
"""
initial_prompt_template = ChatPromptTemplate.from_template(initial_str)
print('initial:',initial_prompt_template.messages[0].prompt.input_variables)
refine_prompt_template = ChatPromptTemplate.from_template(refine_str)
print('refine:',refine_prompt_template.messages[0].prompt.input_variables)

initial: ['output_format', 'text']
refine: ['data', 'output_format', 'text']


### ヘルパー関数

メイン関数で使う関数などを定義した。

`chunk_size`などは、現在はえいやで決めている。\
指示プロンプトと出力Jsonフォーマットのトークン数に基づき、トークン数制約ぎりぎりになるように調整することでチャンク数を抑えられるため、\
今後クラス化して使うときに調整機能をつけようと思う。

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 3072,
    chunk_overlap  = 512,
    length_function = len,
)
chat = ChatOpenAI(temperature=0.)

def load_doc(url):
    response = requests.get(url)
    with open('temp.html','w') as f:
        f.write(response.text)
    loader = UnstructuredHTMLLoader("temp.html")
    data = loader.load()
    return data

def extract_json(text,debug=False):
    patterns = [
        {"pattern": r'```json\n(.*?)```', "group": 1},
        {"pattern": r'\{.*\}', "group": 0},
    ]
    for p in patterns:
        match = re.search(p["pattern"], text, re.DOTALL)
        if match:
            json_string = match.group(p["group"])
            if debug:
                print('json_string:',json_string,sep='\n')
            data_json = json.loads(json_string)
            return data_json
    print('Error:Could not json extracted:',text)
    return {}

### メイン関数

まずはWebページのテキストをLangChainのドキュメントとし、チャンクに分割する。\
１つ目のチャンクにはInitialプロンプト、２つ目以降の各チャンクにはRefineプロンプトを適用する。\
最後にソースURLをデータに加えて終了。

In [4]:
def extact_json_from_url(url,data_format,debug=False):
    doc = load_doc(url)
    docs = text_splitter.split_documents(doc)
    data = {}
    for i,doc in enumerate(docs):
        text = doc.page_content
        if debug:
            print(i+1,'-'*30)
            print(text.replace('\n','').replace(' ','')[:100],'...')
            print('...',text.replace('\n','').replace(' ','')[-100:])
        if i == 0:
            messages = initial_prompt_template.format_messages(
                output_format=json.dumps(data_format,indent=2,ensure_ascii=False),
                text=text
            )
            response = chat(messages)
            data = extract_json(response.content,debug=debug)
        else:
            messages = refine_prompt_template.format_messages(
                output_format=json.dumps(data_format,indent=2,ensure_ascii=False),
                text=text,
                data=json.dumps(data,indent=2,ensure_ascii=False)
            )
            response = chat(messages)
            data = extract_json(response.content,debug=debug)
        if debug:
            print(json.dumps(data,ensure_ascii=False))
    data['url'] = url
    return data

### Example: レシピ

In [5]:
recipe_format = {
    "name": "レシピ名",
    "description": "レシピの説明",
    "cooking_time": "int: minutes",
    "appliances": [
        "str: 調理器具"
    ],
    "serves": "int: 人前",
    "ingredients": [
        {
            "name": "str: 材料名",
            "amount": "str: 分量表記",
        }
    ],
    "steps": {
        "str: 手順番号":"str: 手順の説明"
    }
}

In [8]:
doc = load_doc("https://www.lettuceclub.net/recipe/dish/14360/")


In [10]:
doc[0].page_content

'レタスクラブ\n\nレシピ\n\nカレーおから\n\nカレーおから\n\n時短でラク\n\n275kcal\n\n1.5g\n\n15min\n\n井原裕子さんのレシピ\n\nしっとり煮上げるのがおいしさのコツ\n\n豚こま切れ肉\n\nおから\n\nパプリカ\n\nピーマン\n\n玉ねぎ\n\nその他 豆腐料理\n\nFacebookでシェアする\n\nTwitterでシェアする\n\nPinterestでシェアする\n\nLINEでシェアする\n\n材料コピー\n\n人気のその他 豆腐料理レシピを見る →\n\n↓材\u3000料\n\n↓作り方\n\n材料（２人分）\n\n豚こま切れ肉…100g\n\nおから…100g\n\n赤パプリカ…1/2個\n\nピーマン…２個\n\n玉ねぎ…1/2個\n\n煮汁\n\n・トマトケチャップ…大さじ２\n\n・カレー粉…大さじ１〜１\u30001/2\n\n・酒…大さじ１\n\n・塩…小さじ1/3\n\n・水…130ml\n\n・サラダ油\n\n\n        \n            豚こま切れ肉…100g\n        \n            \n\n作り方\n\nパプリカは縦３等分に切り、横半分に切る。ピーマンは縦半分に切り、さらに縦横それぞれ半分に切る。玉ねぎは縦３等分のくし形に切り、横半分に切る。\n\nフライパンに油大さじ1/2を熱し、豚肉を炒め、肉の色が変わったらを１加えて炒め合わせる。全体に油がまわったら、煮汁の材料を加えて混ぜ、煮立ったらふたをして弱火で３〜４分煮る。\n\nふたを取っておからを加え、全体になじませるように混ぜ合わせたら中火にし、水分をとばしながら約２分炒め煮にする。煮汁が充分に煮立ったところにおからを加え、汁けを吸わせて、しっとり煮上げる。\n\n※カロリー・塩分は１人分での表記になります。\n        \n        \n                ※電子レンジを使う場合は500Wのものを基準としています。600Wなら0.8倍、700Wなら0.7倍の時間で加熱してください。また機種によって差がありますので、様子をみながら加熱してください。\n        \n              ※レシピ作成・表記の基準等は、「レシピについて」をご覧ください。\n

In [6]:
url = "https://www.lettuceclub.net/recipe/dish/14360/"
data = extact_json_from_url(url,recipe_format)
print(json.dumps(data,indent=2,ensure_ascii=False))

{
  "name": "豚こま切れ肉のレシピをもっと見る",
  "description": "",
  "cooking_time": 0,
  "appliances": [],
  "serves": 0,
  "ingredients": [],
  "steps": {},
  "url": "https://www.lettuceclub.net/recipe/dish/14360/"
}


### Example: イベント

In [7]:
event_format = {
    'title': 'str: イベント名',
    'organizer': 'str: 主催者',
    'date': ['str: 日付。形式はYYYY/MM/DD'],
    'location': 'str: 開催場所',
    'description': 'str: イベント概要。400字以内',
    'schedule': ['str: スケジュール。形式はYYYY/MM/DD HH:MM~HH:MM（補足）'],
}

In [8]:
url = "https://www.nagoya-info.jp/event/detail/404/"
data = extact_json_from_url(url,event_format,debug=False)
print(json.dumps(data,indent=2,ensure_ascii=False))

{
  "title": "オアシス２１iセンター体験イベント＜有松・鳴海絞り染め＞",
  "organizer": "オアシス２１iセンター",
  "date": [
    "2023/06/12"
  ],
  "location": "オアシス２１iセンター前（オアシス２１地下1階）",
  "description": "名古屋の伝統産業「有松・鳴海絞り染め」の体験イベント。小さなお子様も参加可能。英語対応も可能。",
  "schedule": [
    "2023/06/12 00:00~00:30"
  ],
  "url": "https://www.nagoya-info.jp/event/detail/404/"
}
